In [1]:
%matplotlib inline

import torch
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# Hyper parameters
num_epochs = 3
batchsize = 64
batchsize_test = 500
learning_rate = 0.001
momentum = 0.5
log_interval = 20
TRAIN_DATA_PATH = "../Data/CATS_DOGS/train"
TEST_DATA_PATH = "../Data/CATS_DOGS/test"

TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(32),
    transforms.CenterCrop(32),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
    ])

train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM_IMG)
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batchsize, shuffle=True,  num_workers=4)

test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=TRANSFORM_IMG)
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=batchsize_test, shuffle=True, num_workers=4) 



In [3]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 10, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = torch.nn.Dropout2d()
        self.fc1 = torch.nn.Linear(500, 50)
        self.fc2 = torch.nn.Linear(50, 1)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 500)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.sigmoid(x)

In [4]:
#Network model
network = Network()

#Stochastic gradient decent
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

criterion = torch.nn.BCELoss()

In [10]:
##Just for testign

test = iter(train_data_loader)
img, target= next(test)

out = network(img)

#test_loss += F.binary_cross_entropy(output, target).item()

out.shape

torch.Size([64, 1])

In [12]:
target.view(-1, 1).shape

torch.Size([64, 1])

In [37]:
#just for testing
conv1 = torch.nn.Conv2d(3, 10, kernel_size=5)
conv2 = torch.nn.Conv2d(10, 20, kernel_size=5)
conv2_drop = torch.nn.Dropout2d()
fc1 = torch.nn.Linear(500, 50)
fc2 = torch.nn.Linear(50, 1)

x = F.relu(F.max_pool2d(conv1(img), 2))
x = F.relu(F.max_pool2d(conv2_drop(conv2(x)), 2))
x = x.view(-1, 500)
x = F.relu(fc1(x))
x = F.dropout(x)
x = fc2(x)
x = F.sigmoid(x)

x.shape

torch.Size([64, 1])

In [20]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_data_loader.dataset) for i in range(num_epochs + 1)]

In [21]:
def test():
    #Sets the module in evaluation mode
    network.eval()
    test_loss = 0
    correct = 0
    
    #dont update dynamic computation graph
    with torch.no_grad():
        #for every example in test
        for data, target in test_data_loader:
            #evaluate the model
            output = network(data)
            
            #acumulate the loss
            test_loss += F.binary_cross_entropy(output, target.view(-1, 1).float()).item()
      
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
        
    test_loss /= len(test_data_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_data_loader.dataset),
    100. * correct / len(test_data_loader.dataset)))

In [22]:
#train

test()

for epoch in range(1, num_epochs+1):
    
    for batch_idx, (data, target) in enumerate(train_data_loader):
        
        #print(f'b {batch}')
        
        #forward pass
        out = network(data)
        
        #Use negative log likelihood loss.
        loss = criterion(out, target.view(-1, 1).float())
        
        #with this gradients are calculated
        loss.backward()
    
        #update gradients
        optimizer.step()

        #Display iteration statistics
        if batch_idx % log_interval == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_data_loader.dataset),
            100. * batch_idx / len(train_data_loader), loss.item()))
        
    test()
    
    


Test set: Avg. loss: 0.0014, Accuracy: 3126/6251 (50%)

Train Epoch: 1 [0/18743 (0%)]	Loss: 0.694885
Train Epoch: 1 [1280/18743 (7%)]	Loss: 0.694191
Train Epoch: 1 [2560/18743 (14%)]	Loss: 0.693920
Train Epoch: 1 [3840/18743 (20%)]	Loss: 0.695292


/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 1 [5120/18743 (27%)]	Loss: 0.679002
Train Epoch: 1 [6400/18743 (34%)]	Loss: 0.689648
Train Epoch: 1 [7680/18743 (41%)]	Loss: 0.692204
Train Epoch: 1 [8960/18743 (48%)]	Loss: 0.686528
Train Epoch: 1 [10240/18743 (55%)]	Loss: 0.697720
Train Epoch: 1 [11520/18743 (61%)]	Loss: 0.684658
Train Epoch: 1 [12800/18743 (68%)]	Loss: 0.678048
Train Epoch: 1 [14080/18743 (75%)]	Loss: 0.682777
Train Epoch: 1 [15360/18743 (82%)]	Loss: 0.685756
Train Epoch: 1 [16640/18743 (89%)]	Loss: 0.682699
Train Epoch: 1 [17920/18743 (96%)]	Loss: 0.630836

Test set: Avg. loss: 0.0013, Accuracy: 3126/6251 (50%)

Train Epoch: 2 [0/18743 (0%)]	Loss: 0.660320
Train Epoch: 2 [1280/18743 (7%)]	Loss: 0.813720
Train Epoch: 2 [2560/18743 (14%)]	Loss: 0.788505
Train Epoch: 2 [3840/18743 (20%)]	Loss: 0.740200
Train Epoch: 2 [5120/18743 (27%)]	Loss: 0.673212
Train Epoch: 2 [6400/18743 (34%)]	Loss: 0.691867
Train Epoch: 2 [7680/18743 (41%)]	Loss: 0.711369
Train Epoch: 2 [8960/18743 (48%)]	Loss: 0.737683
Train Epoc

/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 2 [14080/18743 (75%)]	Loss: 0.720357
Train Epoch: 2 [15360/18743 (82%)]	Loss: 0.682992
Train Epoch: 2 [16640/18743 (89%)]	Loss: 0.669009
Train Epoch: 2 [17920/18743 (96%)]	Loss: 0.661801

Test set: Avg. loss: 0.0014, Accuracy: 3126/6251 (50%)

Train Epoch: 3 [0/18743 (0%)]	Loss: 0.697383
Train Epoch: 3 [1280/18743 (7%)]	Loss: 0.692753


/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 3 [2560/18743 (14%)]	Loss: 0.659104
Train Epoch: 3 [3840/18743 (20%)]	Loss: 0.695865
Train Epoch: 3 [5120/18743 (27%)]	Loss: 0.720312
Train Epoch: 3 [6400/18743 (34%)]	Loss: 0.666602
Train Epoch: 3 [7680/18743 (41%)]	Loss: 0.713859
Train Epoch: 3 [8960/18743 (48%)]	Loss: 0.696500
Train Epoch: 3 [10240/18743 (55%)]	Loss: 0.691055
Train Epoch: 3 [11520/18743 (61%)]	Loss: 0.723630
Train Epoch: 3 [12800/18743 (68%)]	Loss: 0.730478
Train Epoch: 3 [14080/18743 (75%)]	Loss: 0.687728
Train Epoch: 3 [15360/18743 (82%)]	Loss: 0.702145
Train Epoch: 3 [16640/18743 (89%)]	Loss: 0.700803
Train Epoch: 3 [17920/18743 (96%)]	Loss: 0.779970

Test set: Avg. loss: 0.0015, Accuracy: 3126/6251 (50%)



### ToDo

- Checar porque en la validacion del dataset de prueba, parece no estar funcionando
- Checar como se calcula el acuaracy